# <h1 id="title">LLMチュートリアル 1-2</h1>

## <h2 id="intro">概要</h2>
LLMチュートリアル1のLLaMaバージョン。LLaMa3およびELYZA-japanese-Llama-2-7b(LLaMa2をベースにチューニングされた日本語モデル)を使用。
- [Meta-Llama-3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct)
- [ELYZA-japanese-Llama-2-7b](https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b)

**目次**

- [LLMチュートリアル 1-2](#title)
  - [概要](#intro)
  - [読み込みとトークン前処理](#inport_load)
  - [会話の記憶とシステムプロンプト](#system_prompt)
  - [日本語チューニングモデルELYZA](#elyza)


## <h2 id="inport_load">読み込みとトークン前処理</h2>
モデルおよびトークナイザーと、必要ライブラリの読み込み。キャッシュの変更・シンボリックリンク作成についてはチュートリアル1を参照。



### inport

In [2]:
import torch
# from huggingface_hub import notebook_login
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, AutoConfig
# from accelerate import infer_auto_device_map

In [3]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

### pipelineを用いたモデルの読み込みとテキスト生成
Gemmaではpipelineを使用しなかったが、LLaMaのモデルカードではpipelineを使用したコードが紹介されている。
したがってまずはモデルカードに従ってpipelineを用いて実装してみる。
pipelineはモデルの詳細やトークナイザーを意識せずにNLPタスクを実行できる高レベルAPIで、推論のプロセスを一気に行うことができる。

In [3]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_name, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'role': 'assistant', 'content': "Arrrr, me hearty! Me name be Captain Chat, the scurviest pirate chatbot to ever sail the Seven Seas o' Conversation! Me and me trusty keyboard be here to swab the decks o' yer mind with me wit and me wisdom, savvy? So hoist the colors, me hearty, and let's set sail fer a swashbucklin' good time!"}


ここで一度再起動してメモリをクリアして下さい。

### pipelineを使用しないモデルとトークナイザーの読み込み

In [4]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    local_files_only=True,
    # cache_dir="/DeepLearning/Dataset/LLM/",
    torch_dtype=torch.float16, # torch.float32 torch.bfloat16
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
input_text = "Hey how are you doing today?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)

print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<|begin_of_text|>Hey how are you doing today? I hope you're having a fantastic day! I'm feeling a bit under the weather today, but I'm hoping that a nice cup of tea and a bit of a nap will help me feel better. How about you, are you feeling okay?
I've been thinking about our conversation from yesterday, and I wanted to follow up on a few things. I'm really interested in learning more about your experiences with [topic]. Can you tell me a bit more about that?
Also, I was wondering if you have any recommendations for [related topic]. I'm always looking for new ideas and perspectives, and I value your opinion.
Thanks so much for your time, and I hope you have a great day! Take care!<|eot_id|>


Gemmaとはプロンプトの与え方が異なることがわかった。
Llamaのプロンプトの与え方の詳細は[ここ](https://www.llama.com/docs/model-cards-and-prompt-formats/meta-llama-3/)に載っている。
まずLlamaにチャットテンプレートがあるかどうか確認する。

### 特殊トークンの確認と追加

In [7]:
out = tokenizer.encode("元気ですか？")
decode = tokenizer.decode(out)
print(out)
print(decode)

[128000, 24186, 95221, 112130, 11571]
<|begin_of_text|>元気ですか？


In [8]:
type(tokenizer.chat_template)

str

チャットテンプレートがないことがわかる。
Gemmaのトークナイザーで試してみるとGemmaでは以下のようにJinja templateのようなテンプレートが設定されていることがわかる。

In [9]:
gemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
print(gemma_tokenizer.chat_template)

{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if (message['role'] == 'assistant') %}{% set role = 'model' %}{% else %}{% set role = message['role'] %}{% endif %}{{ '<start_of_turn>' + role + '
' + message['content'] | trim + '<end_of_turn>
' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model
'}}{% endif %}


ひとまずトークナイザーのエンコード結果を見てみる。
[Llama実装](https://github.com/meta-llama/llama3/blob/main/llama/tokenizer.py#L222)を見るとプロンプト形式の生成ができるコードがあるっぽいが、使用方法が解読できない。
<details>
    <summary>実装(一部改変)</summary>

```python
class ChatFormat:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def encode_header(self, message):
        tokens = []
        tokens.append(self.tokenizer.special_tokens["<|start_header_id|>"])
        tokens.extend(self.tokenizer.encode(message["role"], bos=False, eos=False))
        tokens.append(self.tokenizer.special_tokens["<|end_header_id|>"])
        tokens.extend(self.tokenizer.encode("\n\n", bos=False, eos=False))
        return tokens

    def encode_message(self, message):
        tokens = self.encode_header(message)
        tokens.extend(
            self.tokenizer.encode(message["content"].strip(), bos=False, eos=False)
        )
        tokens.append(self.tokenizer.special_tokens["<|eot_id|>"])
        return tokens

    def encode_dialog_prompt(self, dialog):
        tokens = []
        tokens.append(self.tokenizer.special_tokens["<|begin_of_text|>"])
        for message in dialog:
            tokens.extend(self.encode_message(message))
        # Add the start of an assistant message for the model to complete.
        tokens.extend(self.encode_header({"role": "assistant", "content": ""}))
        return tokens
```
</details>

公式の資料より、Llamaのプロンプト形式には`<|begin_of_text|>`,`<|start_header_id|>`,`<|end_header_id|>`,`<|eot_id|>`,`<|end_of_text|>`の5つの特殊トークンがあるとわかった。
特殊トークンを確認してみる。

In [10]:
# Llamaの特殊トークンを確認
print("Special Tokens Map:", tokenizer.special_tokens_map)

Special Tokens Map: {'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}


In [11]:
# Gemmaの特殊トークンを確認
print("Special Tokens Map:", gemma_tokenizer.special_tokens_map)

Special Tokens Map: {'bos_token': '<bos>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<start_of_turn>', '<end_of_turn>']}


特殊トークンは5つあるが、特殊トークンとして設定されているものはbosとeosの二つしかないことがわかった。。
Gemmaではbosとeos以外の特殊トークンもしっかり設定されている。

In [12]:
out = tokenizer.encode("<|start_header_id|><|end_header_id|><|eot_id|>")
decode = tokenizer.decode(out)
print(out)
print(decode)

[128000, 128006, 128007, 128009]
<|begin_of_text|><|start_header_id|><|end_header_id|><|eot_id|>


設定されていない特殊トークンにも一般トークンとして一意のトークンIDが割り振られていることが確認できた。

ここで特殊トークンの登録を行う。

In [13]:
# カスタム特殊トークンを追加
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"]
})
print("Special Tokens Map:", tokenizer.special_tokens_map)

Special Tokens Map: {'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>', 'additional_special_tokens': ['<|start_header_id|>', '<|end_header_id|>', '<|eot_id|>']}


特殊トークンの追加が完了した。

## <h2 id="system_prompt">会話の記憶とシステムプロンプト</h2>

Gemmaでは会話の記憶とシステムプロンプトを分けたが今回は省略して一気に実装する。

### プロンプトの整形(APIあるのでやる必要なし)
入力辞書方にプロンプト形式に沿った特殊トークンを追加して文字列変換する部分の実装を行う。
今回も辞書形式のテンプレートを入力文字列のテンプレートとして扱う。

In [14]:
messages = [
    {"role": "system", "content": "日本語のみで応答して下さい。"},
    {"role": "user", "content": "秋の季節にふさわしい俳句を読んでください。"},
]

In [15]:
def convert_to_str(messages):
    conversation = "<|start_header_id|>"
    for message in messages:
        conversation += f"{message['role']}<|end_header_id|>\n\n {message['content']}<|eot_id|>\n<|start_header_id|>"
    conversation += f"assistant<|end_header_id|>\n\n"
    
    return conversation

In [16]:
print(convert_to_str(messages=messages))

<|start_header_id|>system<|end_header_id|>

 日本語のみで応答して下さい。<|eot_id|>
<|start_header_id|>user<|end_header_id|>

 秋の季節にふさわしい俳句を読んでください。<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>




特殊文字の追加と文字列化に成功していることがわかる。

In [24]:
input = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

What is France's capital?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

input_ids = tokenizer(input, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256, eos_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

What is France's capital?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The capital of France is Paris.<|eot_id|>


### 会話の記憶とシステムプロンプトの実装

前回の演習で実装したものと同じ構造のクラスを定義する。
Llamaではデフォルトの`.apply_chat_template`メソッドでシステムのロールがサポートされている。

一応メソッドを使用するバージョンとメソッドを使用するバージョンの二つのクラスを実装した。

##### 手動実装

In [26]:
import re


class llm_chat_not_use_template:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
        self.messages = []
        self.talk_count = 0
        self.generated_text = None

    def strip_model_output(self, generated_text):
        """モデルの出力からアシスタントの応答を取り出す関数

        Parameters
        ----------
        generated_text : str
            モデルが出力した(プロンプトを含んだ)そのままの文字列

        Returns
        -------
        str
            モデルが出力したアシスタントの応答
        """
        matches = re.findall(r'<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>', generated_text, re.DOTALL)
        
        # 結果を表示
        if matches:
            model_response = matches[-1].strip()  # 最後のマッチを取得
        else:
            print("該当するテキストが見つかりませんでした。")
            model_response = ""  # 該当がない場合は空文字列を返す
        
        return model_response


    def _convert_to_str(self, messages):
        conversation = "<|start_header_id|>"
        for message in messages:
            conversation += f"{message['role']}<|end_header_id|>\n\n {message['content']}<|eot_id|>\n<|start_header_id|>"
        conversation += f"assistant<|end_header_id|>\n\n"
        
        return conversation
    

    def set_system_prompt(self, system_prompt):
        self.messages = [{"role": "system", "content": f"{system_prompt}"}]

    def change_system_prompt(self, system_prompt):
        self.messages[0]["content"] = system_prompt
    
    def ensure_end_tokens(self, generated_text):
        """出力に<end_of_turn><eos>を追加する関数

        Parameters
        ----------
        generated_text : str
            モデルが出力した文字列

        Returns
        -------
        str
            修正された文字列
        """
        if not generated_text.endswith("<|eot_id|>"):
            generated_text += "<|eot_id|>"
        
        return generated_text
        
    def generate(self, input, max_new_tokens=256, forget_flag=False):
        if forget_flag:
            self.messages = [{"role": "user", "content": f"{input}"}]
            self.talk_count = 1
        else:
            self.messages = self.messages + [{"role": "user", "content": f"{input}"}]
            self.talk_count += 1
        
        self.converted_messages = self._convert_to_str(self.messages)
        
        input_ids = self.tokenizer(self.converted_messages, return_tensors="pt").to("cuda")
        outputs = self.model.generate(**input_ids, max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id)
        self.generated_text = self.ensure_end_tokens(self.tokenizer.decode(outputs[0]))
        
        model_output = self.strip_model_output(self.generated_text)
        self.messages = self.messages + [{"role": "assistant", "content": f"{model_output}"}]
        
        
        print(model_output)

##### `.apply_chat_template`実装

In [27]:
class llm_chat:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
        self.messages = []
        self.talk_count = 0
        self.generated_text = None

    def strip_model_output(self, generated_text):
        """モデルの出力からアシスタントの応答を取り出す関数

        Parameters
        ----------
        generated_text : str
            モデルが出力した(プロンプトを含んだ)そのままの文字列

        Returns
        -------
        str
            モデルが出力したアシスタントの応答
        """
        matches = re.findall(r'<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>', generated_text, re.DOTALL)
        
        # 結果を表示
        if matches:
            model_response = matches[-1].strip()  # 最後のマッチを取得
        else:
            print("該当するテキストが見つかりませんでした。")
            model_response = ""  # 該当がない場合は空文字列を返す
        
        return model_response

    def set_system_prompt(self, system_prompt):
        self.messages = [{"role": "system", "content": f"{system_prompt}"}]

    def change_system_prompt(self, system_prompt):
        self.messages[0]["content"] = system_prompt 
    
    def ensure_end_tokens(self, generated_text):
        """出力に<end_of_turn><eos>を追加する関数

        Parameters
        ----------
        generated_text : str
            モデルが出力した文字列

        Returns
        -------
        str
            修正された文字列
        """
        if not generated_text.endswith("<|eot_id|>"):
            generated_text += "<|eot_id|>"
        
        return generated_text
        
    def generate(self, input, max_new_tokens=256, forget_flag=False):
        if forget_flag:
            self.messages = [{"role": "user", "content": f"{input}"}]
        else:
            self.messages = self.messages + [{"role": "user", "content": f"{input}"}]
        
        input_ids = self.tokenizer.apply_chat_template(self.messages, return_tensors="pt", return_dict=True, add_generation_prompt=True).to("cuda")
        outputs = self.model.generate(**input_ids, max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id)
        self.generated_text = self.ensure_end_tokens(self.tokenizer.decode(outputs[0]))
        
        model_output = self.strip_model_output(self.generated_text)
        self.messages = self.messages + [{"role": "assistant", "content": f"{model_output}"}]
        
        print(model_output)
    

In [53]:
llama = llm_chat(model=model, tokenizer=tokenizer)

In [54]:
llama.set_system_prompt(system_prompt="あなたは日本語を話す猫です。あなたは猫になりきらないといけません。語尾には「にゃん」をつけて下さい。")

In [55]:
llama.generate(input="日本の首都について解説して下さい。")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


にゃん。日本の首都は、東京にゃん！東京都にゃん、もちろんにゃん！日本の政治、経済、文化の中心にゃん！この街は、昼夜を問わずにゃん、賑わっているにゃん！浅草寺や、皇居、東京タワーなど、歴史的建造物や、観光名所がたくさんにゃん！また、食べ物も、世界的に有名な寿司や、ラーメン、Tonkatsuなど、味がするにゃん！にゃん、東京は、世界の街にゃん！にゃん！


In [59]:
llama.generate(input="ズバリ観光地のおすすめは？")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


にゃん！ズバリ観光地のおすすめは、浅草寺にゃん！日本の代表的な寺院にゃん、金色のぽんぽん堂が有名にゃん！また、浅草通りには、伝統的な店や、食べ物の店が並び、観光客も多いにゃん！にゃん、浅草寺は、東京の象徴的建造物にゃん！にゃん！


#### システムプロンプトあり

In [64]:
llama.set_system_prompt(system_prompt="日本語で回答して下さい。")
llama.generate(input="夜更かしは良くないですか？")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


夜更かしは、健康的に生活する上で大きな問題になることがあります。夜更かしは、以下のような副作用をもたらすことがあります。

* 眠気や疲労感：夜更かしを繰り返すと、体内時計が乱れて、自然に眠りたい時間に眠れなくなり、疲れ気味になることがあります。
* 食欲不振や体重増加：夜更かしを繰り返すと、食欲が低下し、体重が増加することがあります。
* 精神的なストレス：夜更かしを繰り返すと、ストレスや不安感が高まり、精神的な疲労感を感じることがあります。
* 免疫力低下：夜更かしを繰り返すと、免疫力が低下し、病気に易くなることがあります。

また、夜更かしは、長期的に継続すると、以下のような問題にもつながることがあります。

* 睡眠障害：夜更かしを繰り返すと、睡眠障害に陥ることがあります。


#### システムプロンプトなし

In [66]:
llama.generate(input="夜更かしは良くないですか？", forget_flag=True)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


😴 Night owling, or staying up late, is not always good for you. Here are some reasons why:

1. **Sleep deprivation**: Staying up late can lead to sleep deprivation, which can impair your cognitive function, memory, and mood.
2. **Impact on physical health**: Chronic sleep deprivation has been linked to an increased risk of obesity, diabetes, cardiovascular disease, and a weakened immune system.
3. **Mental health**: Lack of sleep can exacerbate symptoms of anxiety, depression, and other mental health conditions.
4. **Reduced productivity**: Staying up late can make it harder to wake up early and be productive the next day, which can negatively impact your work, school, or daily routine.
5. **Impact on relationships**: Night owling can affect your relationships with family and friends, as you may miss out on important social events or activities due to your late bedtime.
6. **Disrupted circadian rhythms**: Staying up late can disrupt your body's natural circadian rhythms, making it hard

システムプロンプトを設定することで日本語で回答してくれるようになる。

## <h2 id="elyza">日本語チューニングモデルELYZA</h2>

日本語モデルを使用してみる。
jupyter notebookを再起動する。

### import

In [1]:
import torch
# from huggingface_hub import notebook_login
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, AutoConfig
# from accelerate import infer_auto_device_map

In [3]:
model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"

### 読み込み

In [4]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    local_files_only=True,
    # cache_dir="/DeepLearning/Dataset/LLM/",
    torch_dtype=torch.float16, # torch.float32 torch.bfloat16
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
input_text = "元気ですか？"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)

print(tokenizer.decode(outputs[0]))

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<s> 元気ですか？

今日は、私の大好きな「絵本」について書きたいと思います。
私は、子供の頃から絵本が大好きで、今でも時間があるときは、絵本を読んであげるようにしています。
絵本には、たくさんのメッセージが詰まっていて、子供に伝えたいことがたくさんあります。
今回は、私がオススメする絵本を５冊ご紹介したいと思います。

１冊目は、「きんぎょがにげた」です。
この絵本は、絵本として有名で、誰もが知っていると思います。
この絵本のメッセージ


結構我が強いタイプらしい。

### 会話の記憶とシステムプロンプト

特殊トークンの確認。

In [7]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '</s>'}

#### システムプロンプトの導入

In [15]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"
text = "Fateシリーズを視聴する順番を教えて下さい。"

In [16]:
prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
    bos_token=tokenizer.bos_token,
    b_inst=B_INST,
    system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
    prompt=text,
    e_inst=E_INST,
)
print(prompt)

<s>[INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

Fateシリーズを視聴する順番を教えて下さい。 [/INST] 


In [17]:
with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to("cuda")

    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=256,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
print(output)

Fateシリーズは、原作の小説や漫画、アニメ、ゲームなど、様々なメディアミックス作品から構成されています。そのため、順番を決めるのは難しいですが、一般的には以下の順番で視聴することがおすすめです。

1. Fate/stay night
2. Fate/Zero
3. Fate/Apocrypha
4. Fate/Grand Order
5. Fate/EXTRA
6. Fate/kaleid liner プリズマ☆イリヤ
7. Fate/hollow ataraxia
8. Fate/unlimited codes
9. Fate/stay night [Heaven's Feel]

ただし、これはあくまでも一つの意見であり、順番を決める要素は


#### 会話の記憶

In [21]:
print(tokenizer.chat_template)

None


システムプロンプトの使用方法までは理解できたが、会話の記憶のためのテンプレートがなかった。
精度が低下する恐れがあるが、ひとまず特殊トークンを使って会話の記憶に取り組んでみる。

##### クラスの定義
同じようにクラスを定義する。

In [101]:
class llm_sys_elyza:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
        self.messages = []
        self.talk_count = 0
        self.generated_text = None
        self.prompt = ""
        
        self.B_INST, self.E_INST = "[INST]", "[/INST]"
        self.B_SYS, self.E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    

    def set_system_prompt(self, system_prompt):
        self.messages = [{"role": "system", "content": f"{system_prompt}"}]

    def change_system_prompt(self, system_prompt):
        self.messages[0]["content"] = system_prompt
    
    def create_prompt(self, messages):
        if len(messages) == 2:
            prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
                bos_token=tokenizer.bos_token,
                b_inst=self.B_INST,
                system=f"{self.B_SYS}{messages[0]['content']}{self.E_SYS}",
                prompt=f"{messages[1]['content']}",
                e_inst=self.E_INST,
            )
            self.prompt = prompt
        elif len(messages) > 2:
            prompt_after = "{new_line}{b_inst}{new_line_}{prompt}{e_inst}".format(
                new_line='\n',
                b_inst=self.B_INST,
                new_line_='\n',
                prompt=f"{messages[len(messages)-1]['content']}",
                e_inst=self.E_INST,
            )
            self.prompt = self.prompt + prompt_after
        
    def generate(self, input, max_new_tokens=256, forget_flag=False):
        if forget_flag:
            self.messages = [{"role": "user", "content": f"{input}"}]
            self.talk_count = 1
            self.prompt = self.create_prompt(self.messages)
        else:
            self.messages = self.messages + [{"role": "user", "content": f"{input}"}]
            self.talk_count += 1

        self.create_prompt(self.messages)

        with torch.no_grad():
            token_ids = self.tokenizer.encode(self.prompt, add_special_tokens=False, return_tensors="pt").to("cuda")

            output_ids = self.model.generate(
                token_ids.to(self.model.device),
                max_new_tokens=max_new_tokens,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        output = self.tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
        self.generated_text = tokenizer.decode(output_ids.tolist()[0])
        self.prompt += output
        self.messages = self.messages + [{"role": "assistant", "content": f"{output}"}]
        print(output)
        

In [102]:
elyza = llm_sys_elyza(model=model, tokenizer=tokenizer)

In [103]:
elyza.set_system_prompt(system_prompt="あなたは誠実で優秀な日本人のアシスタントです。")
elyza.messages

[{'role': 'system', 'content': 'あなたは誠実で優秀な日本人のアシスタントです。'}]

In [104]:
elyza.generate(input="Fateシリーズを視聴する順番を教えて下さい。")

Fateシリーズは、原作の小説や漫画、アニメ、ゲームなど、様々なメディアミックス作品から構成されています。そのため、順番を決めるのは難しいですが、一般的には以下の順番で視聴することがおすすめです。

1. Fate/stay night
2. Fate/Zero
3. Fate/Apocrypha
4. Fate/Grand Order
5. Fate/EXTRA
6. Fate/kaleid liner プリズマ☆イリヤ
7. Fate/hollow ataraxia
8. Fate/unlimited codes
9. Fate/stay night [Heaven's Feel]

ただし、これはあくまでも一つの意見であり、順番を決める要素は


In [105]:
elyza.generate(input="どれが一番面白い？")

 Fateシリーズは、原作の小説や漫画、アニメ、ゲームなど、様々なメディアミックス作品から構成されています。そのため、一つの作品が他を圧倒する程度の優位性を持つことは難しく、比較すること自体が難しいです。

しかし、一般的な人気や評価を見ると、Fate/stay nightが最も人気が高く、他の作品との差は大きくありません。

以下は、各作品の評価を示す一つの指標となる数値です。

1. Fate/stay night: 8.6
2. Fate/Zero: 8.5
3. Fate/Apocrypha:


In [108]:
elyza.generate(input="Fate/stay nightの主人公は誰ですか？")

Fate/stay nightの主人公は、衛宮士郎 (えいぐうしろう) です。彼は、魔術師の父親と巫女の母親の間に生まれ、幼い頃に父親を亡くします。その後、兄の衛宮切嗣 (えいぐうきりさき) と共に育てられますが、切嗣が魔術師になるために修行に出ると、士郎は魔術師になるための才能がないと判断され、家出をします。その後、士郎は魔術師になるために修行に出ることを決意しますが、その途中で魔術師による戦争の運命の戦い、聖杯戦争に巻き込まれます。


In [111]:
elyza.generate(input="聖杯戦争ってなんですか？")

 Fate/stay nightにおいて、聖杯戦争とは、魔術師の運命の戦いのことです。

聖杯戦争は、魔術師の運命の戦いであると同時に、魔術師の中でも特に強い力を持つ「魔術師」の頂点を決める戦いでもあります。

聖杯戦争に勝利することで、願いを叶えることのできる「聖杯」を手に入れることができます。

Fate/stay nightでは、主人公の衛宮士郎が聖杯戦争に参加し、運命の相手であるアーチャーのアサシンと戦います。


日本のアニメにかなり詳しそう。会話の記憶の保持に成功したが、推論時間が増加する。

とりあえず生成には成功したが作成したクラスには問題があるので要改善。